In [127]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler

In [128]:
path = 'data/credit card/default of credit card clients.xls'
file_dataframe = pd.read_excel(path, skiprows=[0])

IGNORE_COLUMNS = ['ID']
CATEGORICAL_COLUMNS = ['EDUCATION', 'MARRIAGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'SEX']
LABEL_COLUMN = 'default payment next month'
NUMERICAL_COLUMNS = list(set(file_dataframe.columns).difference(set(CATEGORICAL_COLUMNS + IGNORE_COLUMNS + [LABEL_COLUMN])))

In [129]:
## Data Cleaning
# Drop NAs
file_dataframe.dropna(inplace=True)
file_dataframe.drop(columns=IGNORE_COLUMNS, inplace=True)

# Rename cloumns
# sex values into explanatory string value
file_dataframe.SEX.replace({1: 'Male', 2: 'Female'}, inplace=True)
# education values into explanatory string value
file_dataframe.EDUCATION.replace({1: 'Graduate school', 2: 'University', 3: 'High school', 4: 'Others'}, inplace=True)
# marriage values into explanatory string value
file_dataframe.MARRIAGE.replace({1: 'Married', 2: 'Single', 3: 'Other'}, inplace=True)

# filter noisy records
file_dataframe = file_dataframe[file_dataframe.EDUCATION.isin(['Graduate school', 'University', 'High school', 'Others'])]
file_dataframe = file_dataframe[file_dataframe.MARRIAGE != 0]
# Convert types
for col in NUMERICAL_COLUMNS:
    file_dataframe[col] = file_dataframe[col].astype(float)
    
for col in CATEGORICAL_COLUMNS:
    file_dataframe[col] = file_dataframe[col].astype('category')

file_dataframe.replace({"no": False, "yes": True}, inplace=True)
file_dataframe[LABEL_COLUMN] = file_dataframe[LABEL_COLUMN].astype(bool)

In [130]:
def one_hot_encode(dataframe):
    encoder = OneHotEncoder(handle_unknown='ignore')
    categorical_columns_df = pd.DataFrame(encoder.fit_transform(dataframe[CATEGORICAL_COLUMNS]).toarray())
    categorical_columns_df.columns = encoder.get_feature_names_out(CATEGORICAL_COLUMNS)
    dataframe = pd.concat([dataframe, categorical_columns_df], axis=1)
    dataframe.dropna(inplace=True) 
    return dataframe
file_dataframe = one_hot_encode(file_dataframe)

In [131]:
def min_max_scaling(dataframe):
    scaler = MinMaxScaler()
    for i in NUMERICAL_COLUMNS:
        dataframe[i] = scaler.fit_transform(dataframe[[i]])
    return dataframe
# file_dataframe = min_max_scaling(file_dataframe)

def standard_scaling(dataframe):
    scaler = StandardScaler()
    for i in NUMERICAL_COLUMNS:
        dataframe[i] = scaler.fit_transform(dataframe[[i]])
    return dataframe
# file_dataframe = standard_scaling(file_dataframe)

In [132]:
# for i in NUMERICAL_COLUMNS:
#     print(i)
#     print(np.any(np.isnan(file_dataframe[i])))
#     print(np.all(np.isfinite(file_dataframe[i])))

# file_dataframe['MARRIAGE'].loc[file_dataframe['MARRIAGE'] == "Married"] = 'm'
# file_dataframe['MARRIAGE'].loc[file_dataframe['MARRIAGE'] == "Single"] = 's'
# file_dataframe.loc[file_dataframe['MARRIAGE'] == "Married", LABEL_COLUMN] = 'm'
# file_dataframe.loc[file_dataframe['MARRIAGE'] == "Single", LABEL_COLUMN] = 's'
# file_dataframe['PAY_0'].loc[file_dataframe['PAY_0'] == -1] = 5
# file_dataframe['PAY_0'].loc[file_dataframe['PAY_0'] == 0] = 10
file_dataframe['MARRIAGE'].replace({'Married' : "m", 'Single': "s"}, inplace=True)
print(file_dataframe.head())
# file_dataframe.to_csv("data/credit card/credit-card-preprocessed.csv", index=False)

   LIMIT_BAL     SEX   EDUCATION MARRIAGE   AGE PAY_0 PAY_2 PAY_3 PAY_4 PAY_5  \
0    20000.0  Female  University        m  24.0     2     2    -1    -1    -2   
1   120000.0  Female  University        s  26.0    -1     2     0     0     0   
2    90000.0  Female  University        s  34.0     0     0     0     0     0   
3    50000.0  Female  University        m  37.0     0     0     0     0     0   
4    50000.0    Male  University        m  57.0    -1     0    -1     0     0   

   ... PAY_6_0  PAY_6_2  PAY_6_3  PAY_6_4  PAY_6_5  PAY_6_6  PAY_6_7  PAY_6_8  \
0  ...     0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1  ...     0.0      1.0      0.0      0.0      0.0      0.0      0.0      0.0   
2  ...     1.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
3  ...     1.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
4  ...     1.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   

   SEX_Female  SEX_Male  
